In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from functools import partial

C:\Miniconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [23]:
n_inputs = 28*28
n_hidden1 = 100
n_hidden2 = 100
n_hidden3 = 100
n_hidden4 = 100
n_hidden5 = 100
n_outputs = 5

# Let's define the placeholders for the inputs and the targets
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [24]:
# Let's create the DNN
he_init = tf.contrib.layers.variance_scaling_initializer()
my_dense_layer = partial(
                    tf.layers.dense, activation=tf.nn.elu, 
                    kernel_initializer=he_init)

with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    hidden2 = my_dense_layer(hidden1, n_hidden2, name="hidden2")
    hidden3 = my_dense_layer(hidden2, n_hidden3, name="hidden3")
    hidden4 = my_dense_layer(hidden3, n_hidden4, name="hidden4")
    hidden5 = my_dense_layer(hidden4, n_hidden5, name="hidden5")
    logits = my_dense_layer(hidden5, n_outputs, activation=None, name="outputs")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")
    
learning_rate = 0.01

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss, name="training_op")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y , 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [8]:
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [5]:
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

In [6]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train1))
        for rnd_indices in np.array_split(rnd_idx, len(X_train1) // batch_size):
            X_batch, y_batch = X_train1[rnd_indices], y_train1[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        # Calculate loss and acc on the validation set to do early stopping
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid1, y: y_valid1})
        if loss_val < best_loss:
            save_path = saver.save(sess, "./my_mnist_model_0_to_4.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    saver.restore(sess, "./my_mnist_model_0_to_4.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test1, y: y_test1})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

0	Validation loss: 0.181033	Best loss: 0.181033	Accuracy: 95.27%
1	Validation loss: 0.125367	Best loss: 0.125367	Accuracy: 97.19%
2	Validation loss: 0.258047	Best loss: 0.125367	Accuracy: 97.30%
3	Validation loss: 0.137535	Best loss: 0.125367	Accuracy: 97.07%
4	Validation loss: 0.120659	Best loss: 0.120659	Accuracy: 97.11%
5	Validation loss: 0.467715	Best loss: 0.120659	Accuracy: 75.76%
6	Validation loss: 0.246886	Best loss: 0.120659	Accuracy: 95.97%
7	Validation loss: 0.234324	Best loss: 0.120659	Accuracy: 97.15%
8	Validation loss: 0.136194	Best loss: 0.120659	Accuracy: 97.58%
9	Validation loss: 0.360426	Best loss: 0.120659	Accuracy: 96.87%
10	Validation loss: 0.769926	Best loss: 0.120659	Accuracy: 74.39%
11	Validation loss: 0.742954	Best loss: 0.120659	Accuracy: 78.81%
12	Validation loss: 0.542269	Best loss: 0.120659	Accuracy: 77.37%
13	Validation loss: 0.487362	Best loss: 0.120659	Accuracy: 79.20%
14	Validation loss: 0.510532	Best loss: 0.120659	Accuracy: 79.24%
15	Validation loss: 

<h1>DNNClassifier</h1>

In [19]:
import numpy as np
import tensorflow as tf
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError

class DNNClassifier(BaseEstimator, ClassifierMixin):
        
    def __init__(self, n_hidden_layers=5, n_neurons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate=0.01, batch_size=20, activation=tf.nn.elu, initializer=he_init,
                 batch_norm_momentum=None, dropout_rate=None, random_state=None):
        """Initialize the DNNClassifier by simply storing all the hyperparameters."""
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None

    def _dnn(self, inputs):
        """Build the hidden layers, with support for batch normalization and dropout."""
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons,
                                     kernel_initializer=self.initializer,
                                     name="hidden%d" % (layer + 1))
            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum,
                                                       training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs

    def _build_graph(self, n_inputs, n_outputs):
        """Build the same model as earlier"""
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)

        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.int32, shape=(None), name="y")

        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None

        dnn_outputs = self._dnn(X)

        logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")

        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                                  logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

        optimizer = self.optimizer_class(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)

        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        # Make the important operations available easily through instance variables
        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver

    def close_session(self):
        if self._session:
            self._session.close()

    def _get_model_params(self):
        """Get all variable values (used for early stopping, faster than saving to disk)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}

    def _restore_model_params(self, model_params):
        """Set all variables to the given values (for early stopping, faster than loading from disk)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)

    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()

        # infer n_inputs and n_outputs from the training set.
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        
        # Translate the labels vector to a vector of sorted class indices, containing
        # integers from 0 to n_outputs - 1.
        # For example, if y is equal to [8, 8, 9, 5, 7, 6, 6, 6], then the sorted class
        # labels (self.classes_) will be equal to [5, 6, 7, 8, 9], and the labels vector
        # will be translated to [3, 3, 4, 0, 2, 1, 1, 1]
        self.class_to_index_ = {label: index
                                for index, label in enumerate(self.classes_)}
        y = np.array([self.class_to_index_[label]
                      for label in y], dtype=np.int32)
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # extra ops for batch normalization
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        # needed in case of early stopping
        max_checks_without_progress = 20
        checks_without_progress = 0
        best_loss = np.infty
        best_params = None
        
        # Now train the model!
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
                for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
                    X_batch, y_batch = X[rnd_indices], y[rnd_indices]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    if self._training is not None:
                        feed_dict[self._training] = True
                    sess.run(self._training_op, feed_dict=feed_dict)
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=feed_dict)
                if X_valid is not None and y_valid is not None:
                    loss_val, acc_val = sess.run([self._loss, self._accuracy],
                                                 feed_dict={self._X: X_valid,
                                                            self._y: y_valid})
                    if loss_val < best_loss:
                        best_params = self._get_model_params()
                        best_loss = loss_val
                        checks_without_progress = 0
                    else:
                        checks_without_progress += 1
                    print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_val, best_loss, acc_val * 100))
                    if checks_without_progress > max_checks_without_progress:
                        print("Early stopping!")
                        break
                else:
                    loss_train, acc_train = sess.run([self._loss, self._accuracy],
                                                     feed_dict={self._X: X_batch,
                                                                self._y: y_batch})
                    print("{}\tLast training batch loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_train, acc_train * 100))
            # If we used early stopping then rollback to the best model found
            if best_params:
                self._restore_model_params(best_params)
            return self

    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X: X})

    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)

    def save(self, path):
        self._saver.save(self._session, path)

In [13]:
dnn_clf = DNNClassifier(random_state=42)
dnn_clf.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

0	Validation loss: 0.190826	Best loss: 0.190826	Accuracy: 96.64%
1	Validation loss: 1.689649	Best loss: 0.190826	Accuracy: 18.73%
2	Validation loss: 1.660114	Best loss: 0.190826	Accuracy: 20.91%
3	Validation loss: 1.778077	Best loss: 0.190826	Accuracy: 22.01%
4	Validation loss: 1.667106	Best loss: 0.190826	Accuracy: 22.01%
5	Validation loss: 1.654532	Best loss: 0.190826	Accuracy: 22.01%
6	Validation loss: 1.680933	Best loss: 0.190826	Accuracy: 18.73%
7	Validation loss: 1.779077	Best loss: 0.190826	Accuracy: 22.01%
8	Validation loss: 1.699482	Best loss: 0.190826	Accuracy: 19.27%
9	Validation loss: 1.767771	Best loss: 0.190826	Accuracy: 20.91%
10	Validation loss: 1.629350	Best loss: 0.190826	Accuracy: 22.01%
11	Validation loss: 1.812643	Best loss: 0.190826	Accuracy: 22.01%
12	Validation loss: 1.675939	Best loss: 0.190826	Accuracy: 18.73%
13	Validation loss: 1.633259	Best loss: 0.190826	Accuracy: 20.91%
14	Validation loss: 1.652904	Best loss: 0.190826	Accuracy: 20.91%
15	Validation loss: 

DNNClassifier(activation=<function elu at 0x000001A7BE2E2268>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<function variance_scaling_initializer.<locals>._initializer at 0x000001A7BC93C730>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [14]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9708114419147694

In [15]:
from sklearn.model_selection import RandomizedSearchCV

def leaky_relu(alpha=0.01):
    def parametrized_leaky_relu(z, name=None):
        return tf.maximum(alpha * z, z, name=name)
    return parametrized_leaky_relu

param_distribs = {
    "n_neurons": [10, 30, 50, 70, 90, 100, 120, 140, 160],
    "batch_size": [10, 50, 100, 500],
    "learning_rate": [0.01, 0.02, 0.05, 0.1],
    "activation": [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
    # you could also try exploring different numbers of hidden layers, different optimizers, etc.
    #"n_hidden_layers": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    #"optimizer_class": [tf.train.AdamOptimizer, partial(tf.train.MomentumOptimizer, momentum=0.95)],
}

rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
                                fit_params={"X_valid": X_valid1, "y_valid": y_valid1, "n_epochs": 1000},
                                random_state=42, verbose=2)
rnd_search.fit(X_train1, y_train1)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] n_neurons=10, batch_size=100, activation=<function elu at 0x000001A7BE2E2268>, learning_rate=0.05 
0	Validation loss: 0.131135	Best loss: 0.131135	Accuracy: 96.36%
1	Validation loss: 0.130872	Best loss: 0.130872	Accuracy: 96.17%
2	Validation loss: 0.153402	Best loss: 0.130872	Accuracy: 96.36%
3	Validation loss: 0.145776	Best loss: 0.130872	Accuracy: 96.76%
4	Validation loss: 0.132235	Best loss: 0.130872	Accuracy: 96.87%
5	Validation loss: 0.140970	Best loss: 0.130872	Accuracy: 96.99%
6	Validation loss: 0.150057	Best loss: 0.130872	Accuracy: 96.25%
7	Validation loss: 0.138547	Best loss: 0.130872	Accuracy: 96.64%
8	Validation loss: 0.129539	Best loss: 0.129539	Accuracy: 96.05%
9	Validation loss: 0.131695	Best loss: 0.129539	Accuracy: 96.44%
10	Validation loss: 0.135688	Best loss: 0.129539	Accuracy: 96.68%
11	Validation loss: 1.787319	Best loss: 0.129539	Accuracy: 19.12%
12	Validation loss: 1.630504	Best loss: 0.129539	Acc

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.2s remaining:    0.0s


0	Validation loss: 0.183487	Best loss: 0.183487	Accuracy: 95.93%
1	Validation loss: 0.106714	Best loss: 0.106714	Accuracy: 96.99%
2	Validation loss: 0.150361	Best loss: 0.106714	Accuracy: 95.78%
3	Validation loss: 0.127387	Best loss: 0.106714	Accuracy: 96.64%
4	Validation loss: 0.109397	Best loss: 0.106714	Accuracy: 96.99%
5	Validation loss: 0.132421	Best loss: 0.106714	Accuracy: 96.29%
6	Validation loss: 0.125783	Best loss: 0.106714	Accuracy: 96.44%
7	Validation loss: 0.242102	Best loss: 0.106714	Accuracy: 94.88%
8	Validation loss: 1.612336	Best loss: 0.106714	Accuracy: 19.27%
9	Validation loss: 1.618384	Best loss: 0.106714	Accuracy: 22.01%
10	Validation loss: 1.614465	Best loss: 0.106714	Accuracy: 19.27%
11	Validation loss: 1.625938	Best loss: 0.106714	Accuracy: 22.01%
12	Validation loss: 1.624183	Best loss: 0.106714	Accuracy: 20.91%
13	Validation loss: 1.636243	Best loss: 0.106714	Accuracy: 19.08%
14	Validation loss: 1.623178	Best loss: 0.106714	Accuracy: 22.01%
15	Validation loss: 

37	Validation loss: 0.096842	Best loss: 0.060442	Accuracy: 98.01%
38	Validation loss: 0.063650	Best loss: 0.060442	Accuracy: 98.51%
39	Validation loss: 0.075109	Best loss: 0.060442	Accuracy: 98.67%
40	Validation loss: 0.070305	Best loss: 0.060442	Accuracy: 98.48%
41	Validation loss: 0.074671	Best loss: 0.060442	Accuracy: 98.71%
42	Validation loss: 0.100321	Best loss: 0.060442	Accuracy: 98.48%
43	Validation loss: 0.093451	Best loss: 0.060442	Accuracy: 98.67%
44	Validation loss: 0.094709	Best loss: 0.060442	Accuracy: 98.16%
45	Validation loss: 0.098850	Best loss: 0.060442	Accuracy: 98.63%
46	Validation loss: 0.093627	Best loss: 0.060442	Accuracy: 98.67%
47	Validation loss: 0.099696	Best loss: 0.060442	Accuracy: 98.32%
48	Validation loss: 0.099620	Best loss: 0.060442	Accuracy: 98.48%
Early stopping!
[CV]  n_neurons=30, batch_size=500, activation=<function relu at 0x000001A7BE30C048>, learning_rate=0.02, total=  20.5s
[CV] n_neurons=30, batch_size=500, activation=<function relu at 0x000001

1	Validation loss: 9.133509	Best loss: 2.372510	Accuracy: 54.18%
2	Validation loss: 3.197797	Best loss: 2.372510	Accuracy: 71.66%
3	Validation loss: 3.479365	Best loss: 2.372510	Accuracy: 72.56%
4	Validation loss: 2.982099	Best loss: 2.372510	Accuracy: 81.04%
5	Validation loss: 1.101210	Best loss: 1.101210	Accuracy: 87.41%
6	Validation loss: 1.374337	Best loss: 1.101210	Accuracy: 83.89%
7	Validation loss: 1.475405	Best loss: 1.101210	Accuracy: 82.92%
8	Validation loss: 470.632080	Best loss: 1.101210	Accuracy: 31.74%
9	Validation loss: 170.168671	Best loss: 1.101210	Accuracy: 57.66%
10	Validation loss: 278.015991	Best loss: 1.101210	Accuracy: 61.18%
11	Validation loss: 120.567238	Best loss: 1.101210	Accuracy: 70.17%
12	Validation loss: 161.904739	Best loss: 1.101210	Accuracy: 72.01%
13	Validation loss: 69.559654	Best loss: 1.101210	Accuracy: 79.24%
14	Validation loss: 91.122765	Best loss: 1.101210	Accuracy: 77.83%
15	Validation loss: 70.470764	Best loss: 1.101210	Accuracy: 85.18%
16	Val

...

13	Validation loss: 0.388907	Best loss: 0.267604	Accuracy: 95.90%
14	Validation loss: 0.276283	Best loss: 0.267604	Accuracy: 96.13%
15	Validation loss: 0.272073	Best loss: 0.267604	Accuracy: 95.90%
16	Validation loss: 0.313487	Best loss: 0.267604	Accuracy: 95.19%
17	Validation loss: 0.217006	Best loss: 0.217006	Accuracy: 96.64%
18	Validation loss: 0.299807	Best loss: 0.217006	Accuracy: 96.44%
19	Validation loss: 0.252067	Best loss: 0.217006	Accuracy: 96.83%
20	Validation loss: 0.219029	Best loss: 0.217006	Accuracy: 96.17%
21	Validation loss: 0.196291	Best loss: 0.196291	Accuracy: 96.95%
22	Validation loss: 0.188056	Best loss: 0.188056	Accuracy: 97.15%
23	Validation loss: 0.173780	Best loss: 0.173780	Accuracy: 97.46%
24	Validation loss: 0.175268	Best loss: 0.173780	Accuracy: 97.30%
25	Validation loss: 0.183506	Best loss: 0.173780	Accuracy: 97.26%
26	Validation loss: 0.165146	Best loss: 0.165146	Accuracy: 97.42%
27	Validation loss: 0.169769	Best loss: 0.165146	Accuracy: 97.38%
28	Validat

58	Validation loss: 0.289582	Best loss: 0.200900	Accuracy: 95.82%
59	Validation loss: 0.241095	Best loss: 0.200900	Accuracy: 96.48%
60	Validation loss: 1.861024	Best loss: 0.200900	Accuracy: 93.75%
61	Validation loss: 0.376959	Best loss: 0.200900	Accuracy: 97.22%
62	Validation loss: 0.288130	Best loss: 0.200900	Accuracy: 97.38%
63	Validation loss: 0.263565	Best loss: 0.200900	Accuracy: 96.91%
64	Validation loss: 0.290306	Best loss: 0.200900	Accuracy: 95.82%
65	Validation loss: 0.218606	Best loss: 0.200900	Accuracy: 96.99%
66	Validation loss: 0.227780	Best loss: 0.200900	Accuracy: 97.58%
67	Validation loss: 0.181837	Best loss: 0.181837	Accuracy: 97.50%
68	Validation loss: 0.205703	Best loss: 0.181837	Accuracy: 97.07%
69	Validation loss: 0.181706	Best loss: 0.181706	Accuracy: 97.65%
70	Validation loss: 0.214718	Best loss: 0.181706	Accuracy: 96.36%
71	Validation loss: 0.172101	Best loss: 0.172101	Accuracy: 97.54%
72	Validation loss: 0.170280	Best loss: 0.170280	Accuracy: 97.38%
73	Validat

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 126.3min finished


0	Validation loss: 0.061664	Best loss: 0.061664	Accuracy: 98.36%
1	Validation loss: 0.055720	Best loss: 0.055720	Accuracy: 98.20%
2	Validation loss: 0.038473	Best loss: 0.038473	Accuracy: 98.59%
3	Validation loss: 0.047277	Best loss: 0.038473	Accuracy: 98.51%
4	Validation loss: 0.050905	Best loss: 0.038473	Accuracy: 98.63%
5	Validation loss: 0.080570	Best loss: 0.038473	Accuracy: 97.97%
6	Validation loss: 0.041476	Best loss: 0.038473	Accuracy: 98.83%
7	Validation loss: 0.044341	Best loss: 0.038473	Accuracy: 98.63%
8	Validation loss: 0.037654	Best loss: 0.037654	Accuracy: 98.87%
9	Validation loss: 0.054451	Best loss: 0.037654	Accuracy: 98.67%
10	Validation loss: 0.032014	Best loss: 0.032014	Accuracy: 99.26%
11	Validation loss: 0.058825	Best loss: 0.032014	Accuracy: 98.79%
12	Validation loss: 0.030225	Best loss: 0.030225	Accuracy: 99.18%
13	Validation loss: 0.051989	Best loss: 0.030225	Accuracy: 98.75%
14	Validation loss: 0.078669	Best loss: 0.030225	Accuracy: 98.48%
15	Validation loss: 

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=DNNClassifier(activation=<function elu at 0x000001A7BE2E2268>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<function variance_scaling_initializer.<locals>._initializer at 0x000001A7BC93C730>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42),
          fit_params={'n_epochs': 1000, 'X_valid': array([[0., 0., ..., 0., 0.],
       [0., 0., ..., 0., 0.],
       ...,
       [0., 0., ..., 0., 0.],
       [0., 0., ..., 0., 0.]], dtype=float32), 'y_valid': array([0, 4, ..., 1, 2], dtype=uint8)},
          iid=True, n_iter=50, n_jobs=1,
          param_distributions={'n_neurons': [10, 30, 50, 70, 90, 100, 120, 140, 160], 'batch_size': [10, 50, 100, 500], 'activation': [<function relu at 0x000001A7BE30C048>, <function elu at 0x000001A7BE2E2268>, <function leaky_relu.<local

In [16]:
rnd_search.best_params_

{'activation': <function tensorflow.python.ops.gen_nn_ops.relu>,
 'batch_size': 500,
 'learning_rate': 0.01,
 'n_neurons': 50}

In [17]:
y_pred = rnd_search.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.989492119089317

In [18]:
rnd_search.best_estimator_.save("./my_best_mnist_model_0_to_4")

In [20]:
# Let's train the best model found, once again, to see how fast it converges
dnn_clf = DNNClassifier(activation=leaky_relu(alpha=0.1), batch_size=500, learning_rate=0.01,
                        n_neurons=50, random_state=42)
dnn_clf.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

0	Validation loss: 0.072400	Best loss: 0.072400	Accuracy: 97.62%
1	Validation loss: 0.067381	Best loss: 0.067381	Accuracy: 97.93%
2	Validation loss: 0.050353	Best loss: 0.050353	Accuracy: 98.40%
3	Validation loss: 0.057750	Best loss: 0.050353	Accuracy: 98.28%
4	Validation loss: 0.052084	Best loss: 0.050353	Accuracy: 98.51%
5	Validation loss: 0.048122	Best loss: 0.048122	Accuracy: 98.59%
6	Validation loss: 0.053389	Best loss: 0.048122	Accuracy: 98.67%
7	Validation loss: 0.046709	Best loss: 0.046709	Accuracy: 98.91%
8	Validation loss: 0.051118	Best loss: 0.046709	Accuracy: 98.71%
9	Validation loss: 0.049088	Best loss: 0.046709	Accuracy: 98.87%
10	Validation loss: 0.064235	Best loss: 0.046709	Accuracy: 98.36%
11	Validation loss: 0.045773	Best loss: 0.045773	Accuracy: 98.79%
12	Validation loss: 0.052007	Best loss: 0.045773	Accuracy: 98.79%
13	Validation loss: 0.054095	Best loss: 0.045773	Accuracy: 98.87%
14	Validation loss: 0.051559	Best loss: 0.045773	Accuracy: 98.87%
15	Validation loss: 

DNNClassifier(activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x000001A7C7909D08>,
       batch_norm_momentum=None, batch_size=500, dropout_rate=None,
       initializer=<function variance_scaling_initializer.<locals>._initializer at 0x000001A7BC93C730>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=50,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [30]:
y_pred = dnn_clf.predict(X_test1)
accuracy_score(y_test1, y_pred)

# Here the accuracy is different because I put leaky_relu in the training instead of relu as the rnd_search_best_params says.
# However, the accuracy is better...Wtf?!

0.9920217941233703

In [23]:
# Let's try to add Batch Normalization
dnn_clf_bn = DNNClassifier(activation=leaky_relu(alpha=0.1), batch_size=500, learning_rate=0.01,
                           n_neurons=50, random_state=42,
                           batch_norm_momentum=0.95)
dnn_clf_bn.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

0	Validation loss: 0.057204	Best loss: 0.057204	Accuracy: 98.24%
1	Validation loss: 0.046775	Best loss: 0.046775	Accuracy: 98.20%
2	Validation loss: 0.038639	Best loss: 0.038639	Accuracy: 98.75%
3	Validation loss: 0.039907	Best loss: 0.038639	Accuracy: 98.75%
4	Validation loss: 0.045205	Best loss: 0.038639	Accuracy: 98.59%
5	Validation loss: 0.034912	Best loss: 0.034912	Accuracy: 98.94%
6	Validation loss: 0.032480	Best loss: 0.032480	Accuracy: 99.02%
7	Validation loss: 0.045129	Best loss: 0.032480	Accuracy: 98.98%
8	Validation loss: 0.044904	Best loss: 0.032480	Accuracy: 98.87%
9	Validation loss: 0.031486	Best loss: 0.031486	Accuracy: 99.10%
10	Validation loss: 0.049647	Best loss: 0.031486	Accuracy: 98.79%
11	Validation loss: 0.045901	Best loss: 0.031486	Accuracy: 98.91%
12	Validation loss: 0.043318	Best loss: 0.031486	Accuracy: 98.79%
13	Validation loss: 0.040476	Best loss: 0.031486	Accuracy: 99.02%
14	Validation loss: 0.043031	Best loss: 0.031486	Accuracy: 99.30%
15	Validation loss: 

DNNClassifier(activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x000001A7C7909C80>,
       batch_norm_momentum=0.95, batch_size=500, dropout_rate=None,
       initializer=<function variance_scaling_initializer.<locals>._initializer at 0x000001A7BC93C730>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=50,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [24]:
y_pred = dnn_clf_bn.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9902704806382565

In [25]:
# Mmm, Batch Normalization did not improve the accuracy. We should try to do another tuning for hyperparameters with BN
# and try again.
# ...

In [26]:
# Now let's go back to our previous model and see how well perform on the training set
y_pred = dnn_clf.predict(X_train1)
accuracy_score(y_train1, y_pred)

0.9992153505956203

In [28]:
# Much better than the test set, so probably it is overfitting the training set. Let's try using dropout
dnn_clf_dropout = DNNClassifier(activation=leaky_relu(alpha=0.1), batch_size=500, learning_rate=0.01,
                                n_neurons=50, random_state=42,
                                dropout_rate=0.5)
dnn_clf_dropout.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

0	Validation loss: 0.502725	Best loss: 0.502725	Accuracy: 84.83%
1	Validation loss: 0.174035	Best loss: 0.174035	Accuracy: 94.72%
2	Validation loss: 0.132589	Best loss: 0.132589	Accuracy: 96.05%
3	Validation loss: 0.122869	Best loss: 0.122869	Accuracy: 96.64%
4	Validation loss: 0.114765	Best loss: 0.114765	Accuracy: 96.87%
5	Validation loss: 0.105026	Best loss: 0.105026	Accuracy: 97.46%
6	Validation loss: 0.096043	Best loss: 0.096043	Accuracy: 97.19%
7	Validation loss: 0.103087	Best loss: 0.096043	Accuracy: 97.62%
8	Validation loss: 0.093458	Best loss: 0.093458	Accuracy: 97.30%
9	Validation loss: 0.096742	Best loss: 0.093458	Accuracy: 97.46%
10	Validation loss: 0.098123	Best loss: 0.093458	Accuracy: 97.58%
11	Validation loss: 0.097413	Best loss: 0.093458	Accuracy: 97.38%
12	Validation loss: 0.089873	Best loss: 0.089873	Accuracy: 97.69%
13	Validation loss: 0.089845	Best loss: 0.089845	Accuracy: 97.62%
14	Validation loss: 0.082723	Best loss: 0.082723	Accuracy: 97.73%
15	Validation loss: 

DNNClassifier(activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x000001A7D0CEEC80>,
       batch_norm_momentum=None, batch_size=500, dropout_rate=0.5,
       initializer=<function variance_scaling_initializer.<locals>._initializer at 0x000001A7BC93C730>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=50,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [29]:
y_pred = dnn_clf_dropout.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9854057209573847

In [31]:
# Dropout doesn't seem to help. As said before, we could try to tune the network with dropout and see what we got.
# ...

<h1>Transfer Learning</h1>

<p>Let's try to reuse the previous model on digits from 5 to 9, using only 100 images per digit!</p>

In [3]:
restore_saver = tf.train.import_meta_graph("./my_best_mnist_model_0_to_4.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
loss = tf.get_default_graph().get_tensor_by_name("loss:0")
Y_proba = tf.get_default_graph().get_tensor_by_name("Y_proba:0")
logits = Y_proba.op.inputs[0]
accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0")

In [4]:
learning_rate = 0.01

output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
# Freeze all the hidden layers
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

In [5]:
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
five_frozen_saver = tf.train.Saver()

In [9]:
X_train2_full = mnist.train.images[mnist.train.labels >= 5]
y_train2_full = mnist.train.labels[mnist.train.labels >= 5] - 5
X_valid2_full = mnist.validation.images[mnist.validation.labels >= 5]
y_valid2_full = mnist.validation.labels[mnist.validation.labels >= 5] - 5
X_test2 = mnist.test.images[mnist.test.labels >= 5]
y_test2 = mnist.test.labels[mnist.test.labels >= 5] - 5

In [10]:
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

In [11]:
X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

In [11]:
import time

n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_best_mnist_model_0_to_4")
    for var in output_layer_vars:
        var.initializer.run()

    t0 = time.time()
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = five_frozen_saver.save(sess, "./my_mnist_model_5_to_9_five_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

    t1 = time.time()
    print("Total training time: {:.1f}s".format(t1 - t0))

with tf.Session() as sess:
    five_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_five_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_best_mnist_model_0_to_4
0	Validation loss: 1.076381	Best loss: 1.076381	Accuracy: 58.00%
1	Validation loss: 0.995478	Best loss: 0.995478	Accuracy: 55.33%
2	Validation loss: 0.954706	Best loss: 0.954706	Accuracy: 64.67%
3	Validation loss: 0.941471	Best loss: 0.941471	Accuracy: 66.67%
4	Validation loss: 0.930525	Best loss: 0.930525	Accuracy: 66.67%
5	Validation loss: 0.912695	Best loss: 0.912695	Accuracy: 71.33%
6	Validation loss: 0.920844	Best loss: 0.912695	Accuracy: 65.33%
7	Validation loss: 0.890324	Best loss: 0.890324	Accuracy: 69.33%
8	Validation loss: 0.916895	Best loss: 0.890324	Accuracy: 64.67%
9	Validation loss: 0.965269	Best loss: 0.890324	Accuracy: 62.67%
10	Validation loss: 0.875946	Best loss: 0.875946	Accuracy: 67.33%
11	Validation loss: 0.953211	Best loss: 0.875946	Accuracy: 65.33%
12	Validation loss: 0.894069	Best loss: 0.875946	Accuracy: 66.67%
13	Validation loss: 0.890609	Best loss: 0.875946	Accuracy: 64.67%
14	Validation l

<p>As we can see, not so good...But of course, we're using 100 images per digit and we only changed the output layer.</p>

In [4]:
# Let's try to reuse only 4 hidden layers instead of 5
n_outputs = 5

restore_saver = tf.train.import_meta_graph("./my_best_mnist_model_0_to_4.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

hidden4_out = tf.get_default_graph().get_tensor_by_name("hidden4_out:0")
logits = tf.layers.dense(hidden4_out, n_outputs, kernel_initializer=he_init, name="new_logits")
Y_proba = tf.nn.softmax(logits)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [5]:
learning_rate = 0.01

output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="new_logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

init = tf.global_variables_initializer()
four_frozen_saver = tf.train.Saver()

In [12]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_best_mnist_model_0_to_4")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = four_frozen_saver.save(sess, "./my_mnist_model_5_to_9_four_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    four_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_four_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_best_mnist_model_0_to_4
0	Validation loss: 1.303004	Best loss: 1.303004	Accuracy: 52.00%
1	Validation loss: 1.046397	Best loss: 1.046397	Accuracy: 54.00%
2	Validation loss: 0.990561	Best loss: 0.990561	Accuracy: 57.33%
3	Validation loss: 0.943077	Best loss: 0.943077	Accuracy: 61.33%
4	Validation loss: 0.924572	Best loss: 0.924572	Accuracy: 61.33%
5	Validation loss: 0.889116	Best loss: 0.889116	Accuracy: 63.33%
6	Validation loss: 0.896398	Best loss: 0.889116	Accuracy: 64.00%
7	Validation loss: 0.858603	Best loss: 0.858603	Accuracy: 66.67%
8	Validation loss: 0.873803	Best loss: 0.858603	Accuracy: 69.33%
9	Validation loss: 0.841726	Best loss: 0.841726	Accuracy: 68.00%
10	Validation loss: 0.862488	Best loss: 0.841726	Accuracy: 67.33%
11	Validation loss: 0.821854	Best loss: 0.821854	Accuracy: 70.67%
12	Validation loss: 0.809741	Best loss: 0.809741	Accuracy: 71.33%
13	Validation loss: 0.810555	Best loss: 0.809741	Accuracy: 70.00%
14	Validation l

<p>Well, a bit better...</p>

In [14]:
# Let's try now to unfreeze the last two layers
learning_rate = 0.01

unfrozen_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[34]|new_logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam3")
training_op = optimizer.minimize(loss, var_list=unfrozen_vars)

init = tf.global_variables_initializer()
two_frozen_saver = tf.train.Saver()

In [15]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    four_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_four_frozen")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = two_frozen_saver.save(sess, "./my_mnist_model_5_to_9_two_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    two_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_two_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_mnist_model_5_to_9_four_frozen
0	Validation loss: 0.853785	Best loss: 0.853785	Accuracy: 72.67%
1	Validation loss: 0.870564	Best loss: 0.853785	Accuracy: 67.33%
2	Validation loss: 0.652311	Best loss: 0.652311	Accuracy: 76.00%
3	Validation loss: 0.698115	Best loss: 0.652311	Accuracy: 78.00%
4	Validation loss: 0.690634	Best loss: 0.652311	Accuracy: 75.33%
5	Validation loss: 0.768487	Best loss: 0.652311	Accuracy: 77.33%
6	Validation loss: 0.848543	Best loss: 0.652311	Accuracy: 80.00%
7	Validation loss: 0.697138	Best loss: 0.652311	Accuracy: 83.33%
8	Validation loss: 0.712297	Best loss: 0.652311	Accuracy: 84.67%
9	Validation loss: 0.779837	Best loss: 0.652311	Accuracy: 80.00%
10	Validation loss: 0.833671	Best loss: 0.652311	Accuracy: 81.33%
11	Validation loss: 0.803134	Best loss: 0.652311	Accuracy: 82.00%
12	Validation loss: 0.954865	Best loss: 0.652311	Accuracy: 78.67%
13	Validation loss: 0.921988	Best loss: 0.652311	Accuracy: 80.00%
14	Valid

<p>Not bad...And what if we unfreeze all the layers?</p>

In [16]:
learning_rate = 0.01

optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam4")
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
no_frozen_saver = tf.train.Saver()

In [17]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    two_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_two_frozen")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = no_frozen_saver.save(sess, "./my_mnist_model_5_to_9_no_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    no_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_no_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_mnist_model_5_to_9_two_frozen
0	Validation loss: 0.753302	Best loss: 0.753302	Accuracy: 79.33%
1	Validation loss: 0.518203	Best loss: 0.518203	Accuracy: 87.33%
2	Validation loss: 0.449906	Best loss: 0.449906	Accuracy: 89.33%
3	Validation loss: 0.387903	Best loss: 0.387903	Accuracy: 92.00%
4	Validation loss: 0.649829	Best loss: 0.387903	Accuracy: 88.00%
5	Validation loss: 0.645186	Best loss: 0.387903	Accuracy: 91.33%
6	Validation loss: 1.225373	Best loss: 0.387903	Accuracy: 88.67%
7	Validation loss: 1.132690	Best loss: 0.387903	Accuracy: 88.67%
8	Validation loss: 0.816616	Best loss: 0.387903	Accuracy: 90.00%
9	Validation loss: 0.769069	Best loss: 0.387903	Accuracy: 90.67%
10	Validation loss: 0.990865	Best loss: 0.387903	Accuracy: 89.33%
11	Validation loss: 1.153069	Best loss: 0.387903	Accuracy: 89.33%
12	Validation loss: 1.028986	Best loss: 0.387903	Accuracy: 90.00%
13	Validation loss: 1.057120	Best loss: 0.387903	Accuracy: 92.00%
14	Valida

In [20]:
# Let's compare this result with a DNN trained from scratch
dnn_clf_5_to_9 = DNNClassifier(n_hidden_layers=4, random_state=42)
dnn_clf_5_to_9.fit(X_train2, y_train2, n_epochs=1000, X_valid=X_valid2, y_valid=y_valid2)

0	Validation loss: 0.803556	Best loss: 0.803556	Accuracy: 71.33%
1	Validation loss: 0.966740	Best loss: 0.803556	Accuracy: 85.33%
2	Validation loss: 1.158966	Best loss: 0.803556	Accuracy: 78.00%
3	Validation loss: 0.615953	Best loss: 0.615953	Accuracy: 88.00%
4	Validation loss: 0.612615	Best loss: 0.612615	Accuracy: 92.00%
5	Validation loss: 0.686583	Best loss: 0.612615	Accuracy: 89.33%
6	Validation loss: 0.804757	Best loss: 0.612615	Accuracy: 89.33%
7	Validation loss: 0.748284	Best loss: 0.612615	Accuracy: 88.00%
8	Validation loss: 0.947906	Best loss: 0.612615	Accuracy: 84.00%
9	Validation loss: 1.652254	Best loss: 0.612615	Accuracy: 89.33%
10	Validation loss: 0.982898	Best loss: 0.612615	Accuracy: 92.67%
11	Validation loss: 1.143990	Best loss: 0.612615	Accuracy: 90.00%
12	Validation loss: 1.167457	Best loss: 0.612615	Accuracy: 92.00%
13	Validation loss: 1.120672	Best loss: 0.612615	Accuracy: 92.00%
14	Validation loss: 1.773683	Best loss: 0.612615	Accuracy: 82.00%
15	Validation loss: 

DNNClassifier(activation=<function elu at 0x000001D68D09B1E0>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<function variance_scaling_initializer.<locals>._initializer at 0x000001D68DFD1268>,
       learning_rate=0.01, n_hidden_layers=4, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [22]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf_5_to_9.predict(X_test2)
accuracy_score(y_test2, y_pred)

0.8996091339230611

<p>Unfortunately in this case transfer learning did not help too much.</p>